In [3]:
import subprocess
import os
import zipfile
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureCollection, FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.geometry import Geometry, Point, Polyline, lengths
import csv

In [4]:
#Logging in
# There are two ways to log in. One would require a client ID, which you might not have access to 
# unless your GIS administrator gives you the right privileges. I had trouble logging in without a client ID, but you might not. 
# Try method 1 first, especially if you don't have a client ID. If that doesn't work... 
# Try getting the right privileges from your admin and proceed with method 2.

In [5]:
# Method 2: Logging in with a client ID
gis_url= "https://amherstcollege.maps.arcgis.com" # url to your org's arcGIS
your_client_id = "Z1lqH7hW52EWwbxZ"               # paste client ID here
gis = GIS(gis_url, client_id=your_client_id)
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://amherstcollege.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Z1lqH7hW52EWwbxZ&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=xfn7gk4DmbdVeFA1x8mr47EZz0LEXV&allow_verification=false


Enter code obtained on signing in using SAML:  ········


Successfully logged in as: tansar27_amherstcollege


In [13]:
# Converts KMLs to shapefiles, zips them, and stores them in a local output folder

kml_folder = r"C:\Users\user\Downloads\idaho_sites_zero_buildings" #path to the folder with all your kmls
output_folder = r"C:\Users\user\Downloads\output_shp" #path to the folder where all the shapefiles will go

def kml_to_shapefile(kml_path, shp_path):
    command = ["ogr2ogr", "-f", "ESRI Shapefile", shp_path, kml_path]
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
        #print(f"Converted '{kml_path}' to '{shp_path}'")
    except subprocess.CalledProcessError as e:
        print(f"Error converting to shapefile: {e}")

def zip_shapefile_components(shp_path):
    folder = os.path.dirname(shp_path)
    basename = os.path.splitext(os.path.basename(shp_path))[0]
    zip_path = os.path.join(folder, basename + ".zip")

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg", ".qix"]:
            filepath = os.path.join(folder, basename + ext)
            if os.path.exists(filepath):
                zipf.write(filepath, os.path.basename(filepath))
                os.remove(filepath)
    #print(f"Zipped to '{zip_path}'")

for file in os.listdir(kml_folder):
    if file.endswith(('.kml', '.kmz')):
        kml_path = os.path.join(kml_folder, file)
        #print(kml_path)
        basename = os.path.splitext(file)[0]
        shp_path = os.path.join(output_folder, basename + ".shp")
        #print(shp_path)
        kml_to_shapefile(kml_path, shp_path)
        zip_shapefile_components(shp_path)

print(f"Completed file conversions. Go check them out in folder: {os.path.basename(output_folder)}.")


Completed file conversions. Go check them out in folder: output_shp.


In [14]:
# Imports the shapefiles from your local output folder to your online ArcGIS account, converts them to feature layers, and publishes them to your webmap 

webmap_search = gis.content.search("title:ce", item_type="Web Map")
webmap_item = webmap_search[0]

webmap_data = webmap_item.get_data()

for zip_shp_file in os.listdir(output_folder):
    zip_shp_path = os.path.join(output_folder, zip_shp_file)
    # print(zip_shp_path)
    name = os.path.basename(zip_shp_file[:-4])
    # print(name)
    existing_layers = gis.content.search(f'title:"{name}"', max_items=1)
    site_exists = len(existing_layers) > 0

    if site_exists:
        print(f"{name} already exists. Skipping.")
        continue
    else:
        item_properties = {"title": str(name), "type": "Shapefile", "tags": "shapefile, layer, ce" } 
        shapefile_item = gis.content.add(item_properties, data=zip_shp_path)
        published_layer = shapefile_item.publish()
        shp_url = published_layer.layers[0].url
        #print(f"Layer published at: {published_layer.url}")
        
        # add to the web map's operational layers
        shp_layer = {"id": shp_url,"title": name,"url": shp_url,"visibility": True,"opacity": 1,"layerType": "ArcGISFeatureLayer"}
        webmap_data["operationalLayers"].append(shp_layer)
        print(f"Added {name} to web map.")
        webmap_item.update(data=webmap_data)
    
print(f"Web map '{webmap_item.title}' successfully updated with CE sites!")

C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Added zero_buildings_Idaho_Falls_100to120 to web map.


C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Added zero_buildings_Idaho_Falls_120to140 to web map.


C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Added zero_buildings_Idaho_Falls_60to80 to web map.


C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Added zero_buildings_Idaho_Falls_80to100 to web map.


C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Added zero_buildings_Idaho_Falls_leq_60 to web map.
Web map 'ce' successfully updated with CE sites!


In [140]:
# Makes rectangular buffers that enclose each CE arm and publishes them to your webmap. 

webmap_search = gis.content.search("title:ce", item_type="Web Map")
webmap_item = webmap_search[0]

webmap_data = webmap_item.get_data()

#change tags and owner as needed 
layers = gis.content.search(f"title:Idaho AND tags: shapefile, layer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

for layer in layers:
    input_layer = layer.layers[0]
    #print(input_layer)
    #print(output_name)
    distance = 200 # enter desired length of your buffer here
                     # 5o meters would result in a extent 40 km x 100 m rectangular extent, 50m on either side of the CE arm.
    units = "Meters"
    end_type="Flat"
    output_name = layer.title.replace(" ","_").replace(".", "_").replace("-","_") + f"_{distance}m_Buffer"
    print(output_name)
    existing_buffers = gis.content.search(f'title:"{output_name}"', max_items=1)
    already_added = len(existing_buffers) > 0

    if already_added:
        print(f"{output_name} already exists. Skipping.")
        continue
    else:
        buffer_result = arcgis.features.use_proximity.create_buffers(input_layer=input_layer,
                                                                     distances=[distance],
                                                                     units=units,
                                                                     end_type=end_type,
                                                                     output_name=output_name)
        buffer_url = buffer_result.layers[0].url
        # add to the web map's operational layers
        new_layer = {"id": buffer_url,"title": output_name,"url": buffer_url,"visibility": True,"opacity": 1,"layerType": "ArcGISFeatureLayer"}
        webmap_data["operationalLayers"].append(new_layer)
        print(f"Added {output_name} to web map.")
        webmap_item.update(data=webmap_data)
    
print(f"Web map '{webmap_item.title}' successfully updated with buffer layers.")


y_arm_157_200m_Buffer


{"cost": 0.001}


Added y_arm_157_200m_Buffer to web map.
Tucson_x_arm_200m_Buffer


{"cost": 0.001}


Added Tucson_x_arm_200m_Buffer to web map.
Idaho_Falls_120to140_200m_Buffer


...

Added Pueblo_x_arm_200m_Buffer to web map.
Idaho_Falls_100to120_200m_Buffer


{"cost": 1.162}


Added Idaho_Falls_100to120_200m_Buffer to web map.
Idaho_Falls_80to100_200m_Buffer


{"cost": 0.404}


Added Idaho_Falls_80to100_200m_Buffer to web map.
Web map 'ce' successfully updated with buffer layers.


In [161]:
# Retrieves the number of buildings, along with each of their latitudess and longitudes, within each of your buffers
# Outputs a csv of buildings

buffers = gis.content.search(f"title:Idaho,Buffer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0" 
buildings = FeatureLayer(buildings_url)

results = []

for buffer in buffers:
    input_buffer = buffer.layers[0]
    
    buffer_features = input_buffer.query(where="1=1", out_fields="*", return_geometry=True).features
    print(buffer)
   
    # for each buffer feature (polygon), count buildings that are within it
    for buffer_feature in buffer_features:
        buffer_geom = buffer_feature.geometry
        buildings_clipped = buildings.query(geometry_filter=intersects(buffer_geom), return_geometry=True)
        buildings_count = len(buildings_clipped.features)
        print("==========")
        print(f"{buffer.title}: {buildings_count} building(s)")
        if buildings_count > 0:
            print("Longitude and latitude for each building:")
            for b in buildings_clipped.features:
                long = b.attributes.get("LONGITUDE")
                lat = b.attributes.get("LATITUDE")
                oid = b.attributes.get("BUILD_ID")
                print(f"Building {oid}: {long}, {lat}")
                results.append([buffer.title, buildings_count, oid, long, lat])

        else:
            print("No buildings found")
            results.append([buffer.title, buildings_count, None, None])
        print("----------")

# outputs a csv of buildings

output_csv = r"C:\Users\user\Downloads\CE_sites_data\Idaho_sites_buildings.csv"

with open(output_csv, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Buffer", "Number of Buildings","Building ID", "Longitude", "Latitude"])
    writer.writerows(results)

print(f"CSV saved to {os.path.abspath(output_csv)}")

<Item title:"Idaho_Falls_60to80_200m_Buffer" type:Feature Layer Collection owner:tansar27_amherstcollege>
Idaho_Falls_60to80_200m_Buffer: 0 building(s)
No buildings found
----------
Idaho_Falls_60to80_200m_Buffer: 17 building(s)
Longitude and latitude for each building:
Building 1714925: -114.13558725415609, 43.131379911585014
Building 1714966: -114.13535690846439, 43.133006115351776
Building 1713723: -113.93276995064433, 42.8865385196695
Building 1713719: -113.9338035334291, 42.88591168987849
Building 1546292: -114.14348616948757, 43.141525186165964
Building 1546329: -114.14205213357623, 43.141652910402364
Building 1714888: -114.12029054537449, 43.11294881025864
Building 1714927: -114.13605070562026, 43.13162054277614
Building 1714933: -114.13579044304005, 43.132081636042095
Building 1714934: -114.13628090665267, 43.13216068337381
Building 1546307: -114.14296555908385, 43.141540935902206
Building 1714889: -114.12059983732459, 43.113007261532985
Building 1714893: -114.12056528901714, 4

KeyboardInterrupt: 

In [43]:
buffers = gis.content.search(
    f"title:Idaho,Buffer,leq AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

summary = []
results = []

for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(where="1=1", out_fields="*", return_geometry=True).features

    # Collect buildings counts for one buffer only
    counts = []
    building_lists = []  # Store actual building features per geometry

    for buffer_feature in buffer_features:
        buffer_geom = buffer_feature.geometry
        buildings_clipped = buildings.query(
            geometry_filter=intersects(buffer_geom),
            return_geometry=True
        )
        count = len(buildings_clipped.features)
        counts.append(count)
        building_lists.append(buildings_clipped.features)  # keep full features
        tot_scores.append(int(buffer_feature.attributes["tot_score"]))  # store here
        
    site_index = 1
    for i in range(0, len(counts), 2):
        count_x = counts[i]
        buildings_x = building_lists[i]
        tot_score_x = tot_scores[i]  # retrieve stored tot_score

        if i + 1 < len(counts):
            count_y = counts[i + 1]
            buildings_y = building_lists[i + 1]
        else:
            count_y = 0
            buildings_y = []

        total_count = count_x + count_y
        site_name = f"{site_index}_{tot_score_x}_{buffer.title[:-12]}"
        print(site_name)
        summary.append([site_name, total_count])

        print("==========")
        print(f"{site_name}: {total_count} building(s)")
        if total_count > 0:
            print("Longitude and latitude for each building:")
            for b in buildings_x + buildings_y:
                long = b.attributes.get("LONGITUDE")
                lat = b.attributes.get("LATITUDE")
                oid = b.attributes.get("BUILD_ID")
                print(f"Building {oid}: {long}, {lat}")
                results.append([site_name, total_count, oid, long, lat])
        else:
            print("No buildings found")
            results.append([site_name, total_count, None, None, None])

        print("----------")
        site_index += 1

output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts_leq.csv"
output_csv_detailed = r"C:\Users\user\Downloads\CE_sites_data\Idaho_counts_coords_leq.csv"

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


with open(output_csv_detailed, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings","Building ID", "Longitude", "Latitude"])
    writer.writerows(results)

print(f"CSV saved to {os.path.abspath(output_csv_detailed)}")

AttributeError: 'Item' object has no attribute 'attributes'

In [56]:
buffers = gis.content.search(
    f"title:Idaho,Buffer AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

summary = []
results = []

for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(where="1=1", out_fields="*", return_geometry=True).features

    # Collect counts for THIS buffer only
    counts = []
    building_lists = []  # Store actual building features per geometry
    tot_scores = []  # store tot_score for each buffer feature
    
    for buffer_feature in buffer_features:
        buffer_geom = buffer_feature.geometry
        buildings_clipped = buildings.query(
            geometry_filter=intersects(buffer_geom),
            return_geometry=True
        )
        count = len(buildings_clipped.features)
        counts.append(count)
        building_lists.append(buildings_clipped.features)  # keep full features
        tot_scores.append(int(buffer_feature.attributes["tot_score"]))  # store here
        
    site_index = 1
    for i in range(0, len(counts), 2):
        count_x = counts[i]
        buildings_x = building_lists[i]
        tot_score_x = tot_scores[i]  # retrieve stored tot_score

        if i + 1 < len(counts):
            count_y = counts[i + 1]
            buildings_y = building_lists[i + 1]
        else:
            count_y = 0
            buildings_y = []

        total_count = count_x + count_y
        site_name = f"{site_index}_{tot_score_x}_{buffer.title[:-12]}"
        print(site_name)
        summary.append([site_name, total_count])

        if total_count > 0:
            for b in buildings_x + buildings_y:
                long = b.attributes.get("LONGITUDE")
                lat = b.attributes.get("LATITUDE")
                oid = b.attributes.get("BUILD_ID")
                results.append([site_name, total_count, oid, long, lat])
        else:
            results.append([site_name, total_count, None, None, None])
        site_index += 1

output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\idaho_building_counts.csv"
output_csv_detailed = r"C:\Users\user\Downloads\CE_sites_data\idaho_counts_coords.csv"

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


with open(output_csv_detailed, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings","Building ID", "Longitude", "Latitude"])
    writer.writerows(results)

print(f"CSV saved to {os.path.abspath(output_csv_detailed)}")

1_79_Idaho_Falls_60to80
2_74_Idaho_Falls_60to80
3_79_Idaho_Falls_60to80
4_75_Idaho_Falls_60to80
5_76_Idaho_Falls_60to80
6_79_Idaho_Falls_60to80
7_79_Idaho_Falls_60to80
8_63_Idaho_Falls_60to80
9_63_Idaho_Falls_60to80
10_68_Idaho_Falls_60to80
11_71_Idaho_Falls_60to80
12_78_Idaho_Falls_60to80
13_70_Idaho_Falls_60to80
14_69_Idaho_Falls_60to80
15_61_Idaho_Falls_60to80
16_63_Idaho_Falls_60to80
17_69_Idaho_Falls_60to80
18_61_Idaho_Falls_60to80
19_60_Idaho_Falls_60to80
20_64_Idaho_Falls_60to80
21_77_Idaho_Falls_60to80
22_65_Idaho_Falls_60to80
23_74_Idaho_Falls_60to80
24_63_Idaho_Falls_60to80
25_72_Idaho_Falls_60to80
26_70_Idaho_Falls_60to80
27_66_Idaho_Falls_60to80
28_66_Idaho_Falls_60to80
29_69_Idaho_Falls_60to80
30_78_Idaho_Falls_60to80
31_65_Idaho_Falls_60to80
32_68_Idaho_Falls_60to80
33_75_Idaho_Falls_60to80
34_77_Idaho_Falls_60to80
35_67_Idaho_Falls_60to80
36_75_Idaho_Falls_60to80
37_74_Idaho_Falls_60to80
38_74_Idaho_Falls_60to80
39_75_Idaho_Falls_60to80
40_69_Idaho_Falls_60to80
1_52_Idah

In [92]:
# Makes circular buffers around each of CE's three end stations and publishes them to your webmap. 

webmap_search = gis.content.search("title:ce", item_type="Web Map")
webmap_item = webmap_search[0]

webmap_data = webmap_item.get_data()


arms = gis.content.search("tags: shapefile, layer AND owner:tansar27_amherstcollege", item_type="Feature Layer", max_items=10000)

for arm in arms:
    input_arm_url = arm.layers[0].url
    input_arm_lyr = FeatureLayer(input_arm_url)
    input_arm = input_arm_lyr.query(where="1=1", out_fields="*", out_sr='4326', return_geometry=True).features
   
    # For each feature in this arm layer, get the last point of last path (end station)
    for i in input_arm:
        arm_geom = i.geometry  
        last_pt_coords = arm_geom["paths"][-1][-1]  # last vertex of last path
        last_pt = Point({"x" : last_pt_coords[0], "y" : last_pt_coords[1], "spatialReference" : {"wkid" : 4326}})
        print(last_pt)
        circle_name = arm.title.replace(" ","_").replace(".", "_").replace("-","_") + f"_{diameter}km_Station_Buffer"
        #circle_buffer = buffer(last_pt, distances=[10], unit=9036, in_sr=4326, geodesic=True)
        #print(circle_buffer)
        feat=Feature(geometry=last_pt)
        #fs = FeatureSet([Feature(geometry=last_pt)])
        #print(fs)
        fc = arcgis.features.FeatureCollection({
            "layers": [
                {
                    "geometryType": "esriGeometryPoint",
                    "objectIdField": "ObjectID",
                    "fields": [
                        {"name": "ObjectID", "type": "esriFieldTypeOID", "alias": "ObjectID"}
                    ],
                    "features": [feat.as_dict]
                }
            ]
        })
        diameter = [10] # enter desired length of your buffer here
                    # 5o meters would result in a extent 40 km x 100 m rectangular extent, 50m on either side of the CE arm.
        units = "Kilometers"
        end_type="Flat"
        existing_circles = gis.content.search(f'title:"{circle_name}"', max_items=1)
        circle_exists = len(existing_circles) > 0
        
        if circle_exists:
            print(f"{circle_name} already exists. Skipping.")
            continue
        else:
            circle_result = arcgis.features.use_proximity.create_buffers(input_layer=fc,
                                                                         distances=distance,
                                                                         units=units,
                                                                         end_type=end_type,
                                                                         output_name=circle_name)
            circle_url = circle_result.layers[0].url
            # add to the web map's operational layers
            circle_layer = {"id": circle_name,"title": circle_name,"url": circle_url,"visibility": True,"opacity": 1,"layerType": "ArcGISFeatureLayer"}
            webmap_data["operationalLayers"].append(circle_layer)
            print(f"Added {circle_name} to web map.")
            webmap_item.update(data=webmap_data)
    
print(f"Web map '{webmap_item.title}' successfully updated with buffer layers.")       

{'x': -120.243894598324, 'y': 43.1027465279585, 'spatialReference': {'wkid': 4326}}
y_arm_157_10km_Station_Buffer already exists. Skipping.
{'x': -112.09551558474, 'y': 32.2453548599743, 'spatialReference': {'wkid': 4326}}
Tucson_x_arm_10km_Station_Buffer already exists. Skipping.
{'x': -112.537971410327, 'y': 39.0478556762782, 'spatialReference': {'wkid': 4326}}
y_arm2500Delta_10km_Station_Buffer already exists. Skipping.
{'x': -111.597689944684, 'y': 32.5331289956269, 'spatialReference': {'wkid': 4326}}
Tucson_y_arm_10km_Station_Buffer already exists. Skipping.
{'x': -113.134273419854, 'y': 38.8333575465742, 'spatialReference': {'wkid': 4326}}
x_arm2500Delta_10km_Station_Buffer already exists. Skipping.
{'x': -102.881846690061, 'y': 38.9970304416953, 'spatialReference': {'wkid': 4326}}
Pueblo_y_arm_10km_Station_Buffer already exists. Skipping.
{'x': -98.0524692850731, 'y': 26.6347173703163, 'spatialReference': {'wkid': 4326}}
x_arm3268_10km_Station_Buffer already exists. Skipping.
{'

In [59]:
circles = gis.content.search(f"title:10km_Station_Buffer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/6" 
local_roads = FeatureLayer(local_roads_url)

road_result = []

for circle in circles:
    input_circle = circle.layers[0]
    
    circle_features = input_circle.query(where="1=1", out_fields="*", out_sr=4326, return_geometry=True).features
   
    # for each buffer feature (polygon), count buildings that intersect it
    for circle_feat in circle_features:
        circle_geom = Geometry(circle_feat.geometry)
        simplified_geoms = arcgis.geometry.generalize(spatial_ref=4326,geometries=[circle_geom], max_deviation=0.001, future=False)
        simplified_geom = simplified_geoms[0]
        circle_coords = circle_geom['rings'][0]  # the list of coordinate pairs
        
        # Print the first coordinate, then skip 5, then next
        sampled_coords = circle_coords[::10]
        if sampled_coords[0] != sampled_coords[-1]:
            sampled_coords.append(sampled_coords[0])
        circle_geom['rings'][0] = sampled_coords
        circle_geometry = Geometry(circle_geom)
        
        roads_in_buffer = local_roads.query(geometry_filter=intersects(circle_geometry), out_sr=4326, return_geometry=True)

        road_geometries = [Geometry(rib.geometry) for rib in roads_in_buffer.features]
        length = lengths(polylines=road_geometries, length_unit=9036, calculation_type='geodesic', spatial_ref = 4326)
        
        total_length = sum(length)
        
        print("==========")
        print(f"{circle.title}: {total_length:.2f} km of roads inside polygon")
        print("----------")

y_arm708Cedar_TRAINS_10km_Station_Buffer: 110.68 km of roads inside polygon
----------
x_arm708Cedar_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
y_arm_157_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
x_arm3268_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
Flagstaff2_x_arm_10km_Station_Buffer: 186.63 km of roads inside polygon
----------
x_arm3270_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
Tucson_y_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
x_arm3269_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
y_arm2500Delta_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Flagstaff2_y_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Flagstaff1_x_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Pueblo_y_arm_10km_Station_Buffer: 136.16 km of roads inside polygon
----------
y_arm3270_10km_Station_Buffer: 55.55 km of roads inside 

In [58]:
circles = gis.content.search(f"title:10km_Station_Buffer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/6" 
local_roads = FeatureLayer(local_roads_url)

road_result = []

for circle in circles:
    input_circle = circle.layers[0]
    
    circle_features = input_circle.query(where="1=1", out_fields="*", out_sr=4326, return_geometry=True).features
   
    # for each buffer feature (polygon), count buildings that intersect it
    for circle_feat in circle_features:
        circle_geom = Geometry(circle_feat.geometry)
        simplified_geoms = arcgis.geometry.generalize(spatial_ref=4326,geometries=[circle_geom], max_deviation=0.001, future=False)
        simplified_geom = simplified_geoms[0]
        circle_coords = circle_geom['rings'][0]  # the list of coordinate pairs
        
        # Print the first coordinate, then skip 5, then next
        sampled_coords = circle_coords[::10]
        if sampled_coords[0] != sampled_coords[-1]:
            sampled_coords.append(sampled_coords[0])
        circle_geom['rings'][0] = sampled_coords
        circle_geometry = Geometry(circle_geom)
        
        roads_in_buffer = local_roads.query(geometry_filter=intersects(simplified_geom), out_sr=4326, return_geometry=True)

        road_geometries = [Geometry(rib.geometry) for rib in roads_in_buffer.features]
        length = lengths(polylines=road_geometries, length_unit=9036, calculation_type='geodesic', spatial_ref = 4326)
        
        total_length = sum(length)
        
        print("==========")
        print(f"{circle.title}: {total_length:.2f} km of roads inside polygon")
        print("----------")

y_arm708Cedar_TRAINS_10km_Station_Buffer: 110.68 km of roads inside polygon
----------
x_arm708Cedar_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
y_arm_157_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
x_arm3268_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
Flagstaff2_x_arm_10km_Station_Buffer: 186.63 km of roads inside polygon
----------
x_arm3270_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
Tucson_y_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
x_arm3269_10km_Station_Buffer: 86.14 km of roads inside polygon
----------
y_arm2500Delta_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Flagstaff2_y_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Flagstaff1_x_arm_10km_Station_Buffer: 0.00 km of roads inside polygon
----------
Pueblo_y_arm_10km_Station_Buffer: 136.16 km of roads inside polygon
----------
y_arm3270_10km_Station_Buffer: 55.55 km of roads inside 

In [26]:
import geopandas as gpd
from shapely.geometry import shape, LineString, Polygon
from arcgis.geometry import Geometry
from arcgis.features import FeatureLayer

# --- 1. Grab your buffer polygons from AGOL (same as you do now) ---
circles = gis.content.search(
    f"title:10km_Station_Buffer AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/6"
local_roads = FeatureLayer(local_roads_url)

for circle in circles:
    input_circle = circle.layers[0]
    circle_features = input_circle.query(where="1=1", out_fields="*", out_sr=4326, return_geometry=True).features

    for circle_feat in circle_features:
        # convert circle feature to shapely polygon
        circle_geom = Geometry(circle_feat.geometry)
        simplified_geoms = arcgis.geometry.generalize(spatial_ref=4326,geometries=[circle_geom], max_deviation=0.001, future=False)
        simplified_geom = simplified_geoms[0]
        poly = shape(simplified_geom)

        # --- 2. Query roads that intersect the buffer polygon ---
        roads_in_buffer = local_roads.query(
            geometry_filter=intersects(simplified_geom),
            out_fields="*",
            return_geometry=True,
            out_sr=4326
        )
        # --- 3. Convert to shapely and CLIP locally ---
        road_shapes = [shape(Geometry(r.geometry)) for r in roads_in_buffer.features]

        clipped_roads = []
        for road in road_shapes:
            clipped = road.intersection(poly)  # clip with shapely
            if not clipped.is_empty:
                clipped_roads.append(clipped)
                print(clipped_roads)
        total_length_km = 0
        # --- 4. Compute total length of clipped roads ---
        for r in clipped_roads:
            print(r.length)
            total_length_km += r.length * 111  # rough conversion degrees→km
        # (if your data are projected, use directly r.length in meters/km instead)

        print("==========")
        print(f"{circle.title}: {total_length_km:.2f} km of *clipped* roads inside polygon")
        print("----------")

[<LINESTRING (-98.121 26.569, -98.121 26.57, -98.121 26.571, -98.12 26.573, -...>]
[<LINESTRING (-98.121 26.569, -98.121 26.57, -98.121 26.571, -98.12 26.573, -...>, <LINESTRING (-98.121 26.569, -98.12 26.57, -98.12 26.571, -98.12 26.572, -98...>]
0.1416625092837728
0.14249636940199115
x_arm3268_10km_Station_Buffer: 31.54 km of *clipped* roads inside polygon
----------
x_arm708Cedar_10km_Station_Buffer: 0.00 km of *clipped* roads inside polygon
----------
[<LINESTRING (-113.827 37.77, -113.827 37.77, -113.826 37.769, -113.825 37.76...>]
[<LINESTRING (-113.827 37.77, -113.827 37.77, -113.826 37.769, -113.825 37.76...>, <LINESTRING (-113.656 37.709, -113.656 37.709, -113.656 37.708, -113.656 37....>]
0.19330414338582816
0.010505001465364108
y_arm708Cedar_TRAINS_10km_Station_Buffer: 22.62 km of *clipped* roads inside polygon
----------
y_arm_157_10km_Station_Buffer: 0.00 km of *clipped* roads inside polygon
----------
Tucson_y_arm_10km_Station_Buffer: 0.00 km of *clipped* roads inside pol

In [15]:


circles = gis.content.search(f"title:10km_Station_Buffer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/6" 
local_roads = FeatureLayer(local_roads_url)

road_result = []

for circle in circles:
    input_circle = circle.layers[0]

    circle_features = input_circle.query(where="1=1", out_fields="*", out_sr=4326, return_geometry=True).features
    # for each buffer feature (polygon), count buildings that intersect it
    for circle_feat in circle_features:
        circle_geom = Geometry(circle_feat.geometry)
        simplified_geoms = arcgis.geometry.generalize(spatial_ref=4326,geometries=[circle_geom], max_deviation=0.001, future=False)
        simplified_geom = simplified_geoms[0]
        circle_coords = circle_geom['rings'][0]  # the list of coordinate pairs
        
        # Print the first coordinate, then skip 5, then next
        sampled_coords = circle_coords[::10]
        if sampled_coords[0] != sampled_coords[-1]:
            sampled_coords.append(sampled_coords[0])
        circle_geom['rings'][0] = sampled_coords
        circle_geometry = Geometry(circle_geom)
        
        #roads_in_buffer = local_roads.query(geometry_filter=intersects(circle_geometry), out_sr=4326, return_geometry=True)
        roads_clipped_name = f"{circle.title} + _clipped" 
        #roads_in_buffer = arcgis.geoanalytics.manage_data.clip_layer(input_layer=local_roads, clip_layer=input_circle)
        roads_in_buffer = arcgis.features.manage_data.overlay_layers(input_layer=input_circle, overlay_layer=local_roads)
        total_length = 0
        for rib in roads_in_buffer:
            print(rib.attributes)
            length_km = rib.attributes['Shape__Length']/1000
            print(length_km)
            total_length += length_km
        
        print("==========")
        print(f"{circle.title}: {total_length:.2f} km of roads inside polygon")
        print("----------")

{"messageCode": "AO_100360", "message": "Empty output generated."}
{"cost": 248.115}


TypeError: 'FeatureCollection' object is not iterable

In [96]:
from arcgis.features import summarize_data 

circles = gis.content.search(f"title:10km_Station_Buffer AND owner:{gis.properties.user.username}", item_type="Feature Layer", max_items=10000)

local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/8" 
local_roads = FeatureLayer(local_roads_url)

road_result = []

for circle in circles:
    input_circle = circle.layers[0].url
    
    local_roads_length = summarize_data.summarize_within(input_circle, local_roads_url, sum_shape=True,shape_units="Kilometers")


{"cost": 16196.757}
{"cost": 16196.757}
C:\Users\user\anaconda3\Lib\site-packages\arcgis\_impl\tools.py:9280: SyntaxWarning: invalid escape sequence '\d'
  """
C:\Users\user\anaconda3\Lib\site-packages\arcgis\_impl\tools.py:18263: SyntaxWarning: invalid escape sequence '\d'
  """


KeyboardInterrupt: 

In [ ]:
buffers = gis.content.search(
    f"title:Idaho,Buffer, leq AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts1.csv"
output_csv_detailed = r"C:\Users\user\Downloads\CE_sites_data\Idaho_counts_coords1.csv"
# --- MAIN ---
summary = []
results = []

# Query all arms (both x and y)
arms_features = arms_layer.query(where="NAME IN ('x arm','y arm')", out_fields="NAME", return_geometry=True).features

# Separate into x and y arms
x_arms = []
y_arms = []

for feat in arms_features:
    start, end = get_endpoints(feat.geometry)
    arm_data = {
        "feature": feat,
        "start": start,
        "end": end
    }
    if feat.attributes["NAME"] == "x arm":
        x_arms.append(arm_data)
    else:
        y_arms.append(arm_data)

# match arms that share an endpoint
site_pairs = []
for x in x_arms:
    for y in y_arms:
        if (x["start"] == y["start"] or
            x["start"] == y["end"] or
            x["end"] == y["start"] or
            x["end"] == y["end"]):
            site_pairs.append((x["feature"], y["feature"]))

# For each site pair, count buildings in both arms' buffers
site_index = 1
for x_feat, y_feat in site_pairs:
    # Query buffer polygons for each arm
    x_buffer = None
    y_buffer = None
    for buffer in buffers:
        input_buffer = buffer.layers[0]
        buf_feats = input_buffer.query(where="1=1", out_fields="*", return_geometry=True).features
        for bf in buf_feats:
            if bf.attributes.get("NAME", "").lower() == "x arm" and bf.attributes.get("OBJECTID") == x_feat.attributes.get("OBJECTID"):
                x_buffer = bf
            elif bf.attributes.get("NAME", "").lower() == "y arm" and bf.attributes.get("OBJECTID") == y_feat.attributes.get("OBJECTID"):
                y_buffer = bf

    if not x_buffer or not y_buffer:
        print(f"Warning: missing buffer for site {site_index}")
        continue

    # Buildings intersecting each arm's buffer
    x_buildings = buildings.query(
        geometry_filter=intersects(x_buffer.geometry),
        return_geometry=True
    ).features

    y_buildings = buildings.query(
        geometry_filter=intersects(y_buffer.geometry),
        return_geometry=True
    ).features

    total_count = len(x_buildings) + len(y_buildings)
    site_name = f"site_{site_index}"
    summary.append([site_name, total_count])

    print("==========")
    print(f"{site_name}: {total_count} building(s)")

    if total_count > 0:
        print("Longitude and latitude for each building:")
        for b in x_buildings + y_buildings:
            long = b.attributes.get("LONGITUDE")
            lat = b.attributes.get("LATITUDE")
            oid = b.attributes.get("BUILD_ID")
            print(f"Building {oid}: {long}, {lat}")
            results.append([site_name, total_count, oid, long, lat])
    else:
        print("No buildings found")
        results.append([site_name, total_count, None, None, None])

    print("----------")
    site_index += 1

# --- SAVE CSVs ---
with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)
print(f"CSV saved to {os.path.abspath(output_csv_summary)}")

with open(output_csv_detailed, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings", "Building ID", "Longitude", "Latitude"])
    writer.writerows(results)
print(f"CSV saved to {os.path.abspath(output_csv_detailed)}")
